## Юнит 9. Профориентация в Data Science
### Skillfactory: DSPR-19
### Проект 7. Возьмёте Бэтмобиль? 

### 1. Чем займёмся в этом проекте?
→ Поздравляем! Вы приступаете к выполнению финального проекта блока Профориентация, в котором вы сможете на практике поработать с задачами из всех трёх направлений — ML, CV и NLP.

За основу возьмем решение, которое нам уже известно, — построенную на табличных данных модель «Предсказание стоимости автомобиля» — и добавим туда модели Deep Learning, которые будут распознавать изображения и обрабатывать текст.

За последние несколько месяцев работы в компании «Старый друг» вы разработали две модели, которые уже успешно крутятся в продакшне и приносят деньги компании и радость сотрудникам (по словам Анны из отдела продаж).

→ Осталось довести свою работу до логичного завершения и, обогатив датасет текстовыми данными из объявлений о продаже, свести все наши модели в единое решение — Multi-inputs сеть. Приступим?

### КАК БУДЕМ РЕШАТЬ ПРОЕКТ?

### ПОСТРОИМ БАЗОВОЕ РЕШЕНИЕ:

- Шаг 1. Построим «наивную» ML-модель на табличных данных, которая предсказывает цену по модели и году выпуска. Это позволит нам задать направление для дальнейших экспериментов. Впоследствии на этом этапе вы можете доработать вашу модель из проекта «Выбираем авто выгодно». 
- Шаг 2. Обработаем и отнормируем признаки и сделаем первую модель на основе градиентного бустинга с помощью CatBoost.
- Шаг 3. Решим эту же задачу с помощью DL (модель NN Tabular) и сравним результаты.
- Шаг 4. Добавим текстовые данные (NLP) и сделаем Multi-Input нейронную сеть для анализа и табличных данных, и текста одновременно.
- Шаг 5. Добавим обработку изображений в Multi-Input нейронную сеть.
- Шаг 6. Осуществим ансамблирование градиентного бустинга и нейронной сети (усредним их предсказания).

### САМОСТОЯТЕЛЬНАЯ ЧАСТЬ

- Ознакомьтесь с критериями оценивания проекта.
- Следуйте нашим рекомендациям, чтобы построить самую лучшую модель.
- Задавайте вопросы одногруппникам и менторам в Slack.
- Загрузите итоговое решение на kaggle и добавьте описание проекта на git.  

На выполнение проекта отводится: две недели.

### ЧТО ВЫ ПОЛУЧИТЕ В РЕЗУЛЬТАТЕ ПРОЕКТА?

- Потренируетесь обрабатывать данные и оптимизировать NLP-модели.
- Пройдетесь по всему циклу разработки комплексной Multi-Inputs модели (от обработки данных до внедрения в продакшн).
- Увидите, как использование blend-решений влияет на целевые метрики.
- Примерите на себя роль специалиста каждого из трёх треков второго года обучения.



— Возьмете Бэтмобиль?  
— Нет. Слишком заметно.  
— Ну тогда возьмите Ламборгини. Это менее заметно.

### 2. Начнём с простой модели

Начнём с разработки baseline нашего решения и посмотрим, как происходит обращение с входящими данными и что нужно получить на выходе. 

По ссылке вы обнаружите baseline. 

СКАЧАТЬ BASELINE (https://github.com/luhakv/cv_ml_nlp/blob/master/baseline-sf-dst-car-price-part2-v6.ipynb)

Итак, наша задача — предсказать стоимость автомобиля по его характеристикам, текстовому описанию или картинке. 

Начнём знакомиться с baseline-решением. Как видим, данные не сильно отличаются от прежних:

bodyType — категориальный  
brand —  категориальный   
color —  категориальный  
description —  текстовый  
engineDisplacement — числовой, представленный как текст  
enginePower — числовой, представленный как текст  
fuelType — категориальный  
mileage —  числовой   
modelDate — числовой  
model_info — категориальный  
name — категориальный, желательно сократить размерность  
numberOfDoors — категориальный  
price — числовой, целевой  
productionDate — числовой  
sell_id — изображение (файл доступен по адресу, основанному на sell_id)  
vehicleConfiguration — не используется (комбинация других столбцов)  
vehicleTransmission — категориальный  
Владельцы — категориальный  
Владение — числовой, представленный как текст  
ПТС — категориальный  
Привод — категориальный  
Руль — категориальный  


Однако кое-что, скорее всего, придётся переделать, поскольку есть дополнительные столбцы. 

→ Переходим к построению самой простой «наивной» модели, которая предсказывает среднюю цену по модели автомобиля и году выпуска. Начинать решение всегда стоит именно с простой модели, чтобы понимать, куда вы движетесь — к улучшению или к ухудшению решения. Это сэкономит вам множество сил и ресурсов. 

Наша модель предсказывает среднее значение по стоимости относительно модели и года выпуска.

